In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from keras.optimizers import Adam,RMSprop,SGD,Adamax,Adagrad,Adadelta,Nadam
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# Load the dataset
df = pd.read_csv('healthvitals.csv')

# Separate features and target variable
X = df.drop(columns=['Fitness Score'])
y = df['Fitness Score']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=28)

# # Preprocess categorical columns (like 'Gender') using one-hot encoding
# categorical_cols = ['Gender']
# ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), categorical_cols)], remainder='passthrough')
# X_train = ct.fit_transform(X_train)
# X_test = ct.transform(X_test)

# Normalize the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# # Build the Neural Network model
# model = keras.Sequential([
#     keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
#     keras.layers.Dense(32, activation='relu'),
#     keras.layers.Dense(1)  # Output layer with a single neuron for regression
# ])
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))


# Compile the model
optimizer = keras.optimizers.Nadam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=180, batch_size=32, validation_split=0.2)

# Make predictions
y_pred = model.predict(X_test).flatten()

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'Mean Squared Error: {mse}')
print(f'R-squared (R2) Score: {r2}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Root Mean Squared Error (RMSE): {rmse}')

In [ ]:
model.save('techne.h5')

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from keras.models import load_model


In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from keras.models import load_model

def predict_fitness_score(input_data):
    # Load the pre-trained model
    model = load_model('techne_ANN.h5')

    # Load the dataset
    df = pd.read_csv('health_vitals_data.csv')

    # Separate features and target variable
    X = df.drop(columns=['Fitness Score'])
    y = df['Fitness Score']

    # Preprocess categorical columns (like 'Gender') using one-hot encoding
    categorical_cols = ['Gender']
    ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), categorical_cols)], remainder='passthrough')
    X = ct.fit_transform(X)

    # Normalize the data
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)

    # Preprocess the input data
    input_data = pd.DataFrame(input_data, columns=df.drop(columns=['Fitness Score']).columns)
    input_data = ct.transform(input_data)
    input_data = scaler.transform(input_data)

    # Make predictions using the trained model
    predictions = model.predict(input_data).flatten()
    return predictions


In [ ]:
#  USER INPUT FUNCTION


class HealthParams:
    def __init__(self, value):
        self.value = value
        self.next = None

class HealthVitalsLinkedList:
    def __init__(self):
        self.head = None

    def prepend(self, value):
        new_node = HealthParams(value)
        new_node.next = self.head
        self.head = new_node

    def get_recent_fit_scores(self, n):
        recent_fit_scores = []
        current_node = self.head
        while current_node and n > 0:
            recent_fit_scores.append(current_node.value)
            current_node = current_node.next
            n -= 1
        return recent_fit_scores

def get_float_input(prompt):
    while True:
        try:
            value = float(input(prompt))
            return value
        except ValueError:
            print("Invalid input. Please enter a valid number.")

def get_user_input(heart_rate_list, sleep_duration_list, systolic_bp_list, diastolic_bp_list, stress_level_list):
    try:
        hr = get_float_input("Enter Heart Rate values: ")
        heart_rate_list.prepend(hr)

        sleep = get_float_input("Enter Sleep Duration values: ")
        sleep_duration_list.prepend(sleep)

        sys_bp = get_float_input("Enter Systolic Blood Pressure values: ")
        systolic_bp_list.prepend(sys_bp)

        dias_bp = get_float_input("Enter Diastolic Blood Pressure values: ")
        diastolic_bp_list.prepend(dias_bp)

        stress = get_float_input("Enter Stress Level values: ")
        stress_level_list.prepend(stress)

        # Return user input as a dictionary
        user_input = {
            'Heart Rate': hr,
            'Stress Level': stress,
            'Systolic': sys_bp,
            'Diastolic': dias_bp,
            'Sleep Duration': sleep,
        }
        return user_input
    except ValueError:
        print("Invalid input. Please enter a valid number.")

def main():
    # Create HealthVitalsLinkedList instances
    heart_rate_list = HealthVitalsLinkedList()
    sleep_duration_list = HealthVitalsLinkedList()
    systolic_bp_list = HealthVitalsLinkedList()
    diastolic_bp_list = HealthVitalsLinkedList()
    stress_level_list = HealthVitalsLinkedList()
    fitness_score_list = HealthVitalsLinkedList()

    # Get user input and store in respective linked lists
    user_input = get_user_input(heart_rate_list, sleep_duration_list, systolic_bp_list, diastolic_bp_list, stress_level_list)
    # prediction = predict_fitness_score([user_input])
    # print("Predicted Fitness Score:", prediction[0])

    # Display the linked lists for each health vital
    def display_linked_list(linked_list, health_vital_name):
        current = linked_list.head
        while current:
            print(f"{health_vital_name}: {current.value}")
            current = current.next
        print()

    display_linked_list(heart_rate_list, "Heart Rate")
    display_linked_list(sleep_duration_list, "Sleep Duration")
    display_linked_list(systolic_bp_list, "Systolic Blood Pressure")
    display_linked_list(diastolic_bp_list, "Diastolic Blood Pressure")
    display_linked_list(stress_level_list, "Stress Level")
    display_linked_list(fitness_score_list, "Fitness Score")

if __name__ == "__main__":
    main()


In [ ]:
df =pd.read_csv('health_vitals_data.csv')

In [ ]:
df

,Gender,Age,Sleep Duration,Stress Level,Heart Rate,Daily Steps,Systolic,Diastolic,Fitness Score
0,Male,27,0.111111,0.6,0.571429,4200,0.407407,0.40,0.486243
1,Male,28,0.148148,1.0,0.476190,10000,0.370370,0.25,0.511376
2,Male,28,0.148148,1.0,0.476190,10000,0.370370,0.25,0.511376
3,Male,28,0.037037,1.0,0.952381,3000,0.925926,0.75,0.832011
4,Male,28,0.037037,1.0,0.952381,3000,0.925926,0.75,0.832011
...,...,...,...,...,...,...,...,...,...
369,Female,59,0.851852,0.0,0.142857,7000,0.925926,1.00,0.683598
370,Female,59,0.814815,0.0,0.142857,7000,0.925926,1.00,0.672487
371,Female,59,0.851852,0.0,0.142857,7000,0.925926,1.00,0.683598
372,Female,59,0.851852,0.0,0.142857,7000,0.925926,1.00,0.683598


In [ ]:
# Perform one-hot encoding for categorical features (if applicable)
df_encoded = pd.get_dummies(df, columns=['Gender'], drop_first=True)

In [ ]:
df_encoded.to_csv("healthvitals.csv", index=False)

In [ ]:
df1 =pd.read_csv('healthvitals.csv')

In [ ]:
df1


,Age,Sleep Duration,Stress Level,Heart Rate,Daily Steps,Systolic,Diastolic,Fitness Score,Gender_Male
0,27,0.111111,0.6,0.571429,4200,0.407407,0.40,0.486243,1
1,28,0.148148,1.0,0.476190,10000,0.370370,0.25,0.511376,1
2,28,0.148148,1.0,0.476190,10000,0.370370,0.25,0.511376,1
3,28,0.037037,1.0,0.952381,3000,0.925926,0.75,0.832011,1
4,28,0.037037,1.0,0.952381,3000,0.925926,0.75,0.832011,1
...,...,...,...,...,...,...,...,...,...
369,59,0.851852,0.0,0.142857,7000,0.925926,1.00,0.683598,0
370,59,0.814815,0.0,0.142857,7000,0.925926,1.00,0.672487,0
371,59,0.851852,0.0,0.142857,7000,0.925926,1.00,0.683598,0
372,59,0.851852,0.0,0.142857,7000,0.925926,1.00,0.683598,0


In [ ]:
df1.describe()


,Age,Sleep Duration,Stress Level,Heart Rate,Daily Steps,Systolic,Diastolic,Fitness Score,Gender_Male
count,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000
mean,42.184492,0.493365,0.477005,0.245989,6816.844920,0.501981,0.482487,0.514101,0.505348
std,8.673133,0.294688,0.354905,0.196937,1617.915679,0.286967,0.308081,0.149419,0.500641
min,27.000000,0.000000,0.000000,0.000000,3000.000000,0.000000,0.000000,0.227302,0.000000
25%,35.250000,0.222222,0.200000,0.142857,5600.000000,0.370370,0.250000,0.424074,0.000000
50%,43.000000,0.518519,0.400000,0.238095,7000.000000,0.555556,0.500000,0.500635,1.000000
75%,50.000000,0.740741,0.800000,0.333333,8000.000000,0.740741,0.750000,0.584762,1.000000
max,59.000000,1.000000,1.000000,1.000000,10000.000000,1.000000,1.000000,0.893333,1.000000
